In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_ProActiv.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg_dist import VGG_Dist
    net = nn.DataParallel(VGG_Dist(nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_Dist(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv4): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [50, 50, 50, 50]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [11]:
count = 0

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 14.850
[TEST] Acc: 16.190
Epoch: 1
[TRAIN] Acc: 17.494
[TEST] Acc: 16.750
Epoch: 2
[TRAIN] Acc: 18.588
[TEST] Acc: 22.250
Epoch: 3
[TRAIN] Acc: 21.028
[TEST] Acc: 22.350
Epoch: 4
[TRAIN] Acc: 24.364
[TEST] Acc: 27.290
Epoch: 5
[TRAIN] Acc: 26.538
[TEST] Acc: 29.720
Epoch: 6
[TRAIN] Acc: 30.364
[TEST] Acc: 31.840
Epoch: 7
[TRAIN] Acc: 33.790
[TEST] Acc: 45.220
Epoch: 8
[TRAIN] Acc: 42.310
[TEST] Acc: 44.040
Epoch: 9
[TRAIN] Acc: 49.844
[TEST] Acc: 51.660
Epoch: 10
[TRAIN] Acc: 54.200
[TEST] Acc: 51.750
Epoch: 11
[TRAIN] Acc: 57.168
[TEST] Acc: 57.040
Epoch: 12
[TRAIN] Acc: 59.622
[TEST] Acc: 68.020
Epoch: 13
[TRAIN] Acc: 62.484
[TEST] Acc: 68.400
Epoch: 14
[TRAIN] Acc: 65.564
[TEST] Acc: 67.030
Epoch: 15
[TRAIN] Acc: 68.110
[TEST] Acc: 68.210
Epoch: 16
[TRAIN] Acc: 70.466
[TEST] Acc: 72.060
Epoch: 17
[TRAIN] Acc: 72.238
[TEST] Acc: 72.670
Epoch: 18
[TRAIN] Acc: 74.224
[TEST] Acc: 76.260
Epoch: 19
[TRAIN] Acc: 75.978
[TEST] Acc: 77.330
Epoch: 20
[TRAIN] Acc: 77.494
[TEST] Ac

[TEST] Acc: 87.880
Epoch: 167
[TRAIN] Acc: 99.500
[TEST] Acc: 87.870
Epoch: 168
[TRAIN] Acc: 99.544
[TEST] Acc: 87.480
Epoch: 169
[TRAIN] Acc: 99.502
[TEST] Acc: 87.660
Epoch: 170
[TRAIN] Acc: 99.534
[TEST] Acc: 87.670
Epoch: 171
[TRAIN] Acc: 99.518
[TEST] Acc: 87.540
Epoch: 172
[TRAIN] Acc: 99.504
[TEST] Acc: 87.750
Epoch: 173
[TRAIN] Acc: 99.582
[TEST] Acc: 87.690
Epoch: 174
[TRAIN] Acc: 99.500
[TEST] Acc: 87.690
Epoch: 175
[TRAIN] Acc: 99.488
[TEST] Acc: 87.650
Epoch: 176
[TRAIN] Acc: 99.472
[TEST] Acc: 87.670
Epoch: 177
[TRAIN] Acc: 99.506
[TEST] Acc: 87.710
Epoch: 178
[TRAIN] Acc: 99.526
[TEST] Acc: 87.410
Epoch: 179
[TRAIN] Acc: 99.558
[TEST] Acc: 87.540
Epoch: 180
[TRAIN] Acc: 99.524
[TEST] Acc: 87.420
Epoch: 181
[TRAIN] Acc: 99.540
[TEST] Acc: 87.430
Epoch: 182
[TRAIN] Acc: 99.496
[TEST] Acc: 87.950
Epoch: 183
[TRAIN] Acc: 99.514
[TEST] Acc: 87.750
Epoch: 184
[TRAIN] Acc: 99.524
[TEST] Acc: 87.610
Epoch: 185
[TRAIN] Acc: 99.554
[TEST] Acc: 87.790
Epoch: 186
[TRAIN] Acc: 99.556


In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_ProActiv.pth'
resume = False

In [ ]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 1.026
[TEST] Acc: 0.930
Epoch: 1
[TRAIN] Acc: 0.938
[TEST] Acc: 0.970
Epoch: 2
[TRAIN] Acc: 0.958
[TEST] Acc: 0.990
Epoch: 3
[TRAIN] Acc: 1.028
[TEST] Acc: 0.900
Epoch: 4
[TRAIN] Acc: 0.952
[TEST] Acc: 1.010
Epoch: 5
[TRAIN] Acc: 0.944
[TEST] Acc: 0.860
Epoch: 6
[TRAIN] Acc: 1.002
[TEST] Acc: 1.040
Epoch: 7
[TRAIN] Acc: 1.024
[TEST] Acc: 0.970
Epoch: 8
[TRAIN] Acc: 1.050
[TEST] Acc: 1.060
Epoch: 9
[TRAIN] Acc: 1.014
[TEST] Acc: 0.830
Epoch: 10
[TRAIN] Acc: 0.970
[TEST] Acc: 0.960
Epoch: 11
[TRAIN] Acc: 1.022
[TEST] Acc: 1.090
Epoch: 12
[TRAIN] Acc: 0.996
[TEST] Acc: 1.010
Epoch: 13
[TRAIN] Acc: 0.992
[TEST] Acc: 1.090
Epoch: 14
[TRAIN] Acc: 0.970
[TEST] Acc: 0.890
Epoch: 15
[TRAIN] Acc: 0.942
[TEST] Acc: 1.020
Epoch: 16
[TRAIN] Acc: 1.054
[TEST] Acc: 0.970
Epoch: 17
[TRAIN] Acc: 1.002
[TEST] Acc: 1.050
Epoch: 18
[TRAIN] Acc: 1.000
[TEST] Acc: 1.050
Epoch: 19
[TRAIN] Acc: 1.020
[TEST] Acc: 1.200
Epoch: 20
[TRAIN] Acc: 0.952
[TEST] Acc: 1.040
Epoch: 21
[TRAIN] Acc: 0.870
[TE